In [ ]:
# #!/usr/bin/env python
# # -*- coding: utf-8 -*- 


# """--------------------------------------------------------------------
# REINFORCEMENT LEARNING

# Started on the 25/08/2017


# theo.alves.da.costa@gmail.com
# https://github.com/theolvs
# ------------------------------------------------------------------------
# """

# import os
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np 
# import sys
# import random
# import time
# import random
# import numpy as np

# from keras.models import Sequential
# from keras.layers import Dense
# from keras.optimizers import Adam

# from rl.memory import Memory

# class Agent(object):
#     def __init__(self):
#         pass


#     def expand_state_vector(self,state):
#         if len(state.shape) == 1 or len(state.shape)==3:
#             return np.expand_dims(state,axis = 0)
#         else:
#             return state



#     def remember(self,*args):
#         self.memory.save(args)

# class QAgent(Agent):
#     def __init__(self,states_size,actions_size,epsilon = 1.0,epsilon_min = 0.01,epsilon_decay = 0.999,gamma = 0.95,lr = 0.8):
#         self.states_size = states_size
#         self.actions_size = actions_size
#         self.epsilon = epsilon
#         self.epsilon_min = epsilon_min
#         self.epsilon_decay = epsilon_decay
#         self.gamma = gamma
#         self.lr = lr
#         self.Q = self.build_model(states_size,actions_size)


#     def build_model(self,states_size,actions_size):
#         Q = np.zeros([states_size,actions_size])
#         return Q


#     def train(self,s,a,r,s_next):
#         self.Q[s,a] = self.Q[s,a] + self.lr * (r + self.gamma*np.max(self.Q[s_next,a]) - self.Q[s,a])

#         if self.epsilon > self.epsilon_min:
#             self.epsilon *= self.epsilon_decay


#     def act(self,s):

#         q = self.Q[s,:]

#         if np.random.rand() > self.epsilon:
#             a = np.argmax(q)
#         else:
#             a = np.random.randint(self.actions_size)

#         return a

# class DQNAgentTSP(Agent):
#     def __init__(self,states_size,actions_size,epsilon = 1.0,epsilon_min = 0.01,epsilon_decay = 0.995,gamma = 0.95,lr = 0.001,low = 0,high = 1,max_memory = 2000,observation_type = "discrete"):
#         assert observation_type in ["discrete","continuous"]
#         self.states_size = states_size
#         self.actions_size = actions_size
#         self.memory = Memory(max_memory = max_memory)
#         self.epsilon = epsilon
#         self.low = low
#         self.high = high
#         self.observation_type = observation_type
#         self.epsilon_min = epsilon_min
#         self.epsilon_decay = epsilon_decay
#         self.gamma = gamma
#         self.lr = lr
#         self.model = self.build_model(states_size,actions_size)

#     def build_model(self,states_size,actions_size):
#         model = Sequential()
#         model.add(Dense(24,input_dim = states_size,activation = "relu"))
#         model.add(Dense(24,activation = "relu"))
#         model.add(Dense(actions_size,activation = "linear"))
#         model.compile(loss='mse',
#                       optimizer=Adam(lr=self.lr))
#         return model

#     def train(self,batch_size = 32):
#         if len(self.memory.cache) > batch_size:
#             batch = random.sample(self.memory.cache, batch_size)
#         else:
#             batch = self.memory.cache

#         for state,action,reward,next_state,done in batch:
#             state = self.expand_state_vector(state)
#             next_state = self.expand_state_vector(next_state)


#             targets = self.model.predict(state)

#             if not done:
#                 target = reward + self.gamma * np.max(self.model.predict(next_state))
#             else:
#                 target = reward

#             targets[0][action] = target

#             self.model.fit(state,targets,epochs = 1,verbose = 0)


#         if self.epsilon > self.epsilon_min:
#             self.epsilon *= self.epsilon_decay

#     def act(self,state):
#         state = self.expand_state_vector(state)


#         if np.random.rand() > self.epsilon:
#             q = self.model.predict(state)

#             if self.observation_type == "discrete":
#                 a = np.argmax(q[0])
#             elif self.observation_type == "continuous":
#                 a = np.squeeze(np.clip(q,self.low,self.high))

#         else:
#             if self.observation_type == "discrete":
#                 a = np.random.randint(self.actions_size)
#             elif self.observation_type == "continuous":
#                 a = np.random.uniform(self.low,self.high,self.actions_size)
#         return a 

# class DeliveryQAgent(QAgent):

#     def __init__(self,*args,**kwargs):
#         super().__init__(*args,**kwargs)
#         self.reset_memory()

#     def act(self,s):

#         # Get Q Vector
#         q = np.copy(self.Q[s,:])

#         # Avoid already visited states
#         q[self.states_memory] = -np.inf

#         if np.random.rand() > self.epsilon:
#             a = np.argmax(q)
#         else:
#             a = np.random.choice([x for x in range(self.actions_size) if x not in self.states_memory])

#         return a


#     def remember_state(self,s):
#         self.states_memory.append(s)

#     def reset_memory(self):
#         self.states_memory = []



# def run_n_episodes(env,agent,name="training.gif",n_episodes=1000,render_each=10,fps=10):

#     # Store the rewards
#     rewards = []
#     imgs = []

#     # Experience replay
#     for i in tqdm_notebook(range(n_episodes)):

#         # Run the episode
#         env,agent,episode_reward = run_episode(env,agent,verbose = 0)
#         rewards.append(episode_reward)
        
#         if i % render_each == 0:
#             img = env.render(return_img = True)
#             imgs.append(img)

#     # Show rewards
#     plt.figure(figsize = (15,3))
#     plt.title("Rewards over training")
#     plt.plot(rewards)
#     plt.show()

#     # Save imgs as gif
#     imageio.mimsave(name,imgs,fps = fps)

#     return env,agent


# class DeliveryQAgent(QAgent):

#     def __init__(self,*args,**kwargs):
#         super().__init__(*args,**kwargs)
#         self.reset_memory()

#     def act(self,s):

#         # Get Q Vector
#         q = np.copy(self.Q[s,:])

#         # Avoid already visited states
#         q[self.states_memory] = -np.inf

#         if np.random.rand() > self.epsilon:
#             a = np.argmax(q)
#         else:
#             a = np.random.choice([x for x in range(self.actions_size) if x not in self.states_memory])

#         return a


#     def remember_state(self,s):
#         self.states_memory.append(s)

#     def reset_memory(self):
#         self.states_memory = []


In [ ]:
#TSP Environment
import gym
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist



# Define the environment
class TSPEnv(gym.Env):
  def __init__(self, n_cities=4, show_debug_data = False):
    self.n_cities = n_cities
    # self.xy = (np.random.rand(self.n_cities,2)*100).round(2)
    self.xy = np.array([[20.01, 80.01], [10.01, 50.01],[50.01, 30.01],[80.01 , 80.01]])
    self.x=self.xy[:,0]
    self.y=self.xy[:,1]
    self.step_counter = 0
    self.show_debug_data = show_debug_data
    self._array_visited = np.zeros(self.n_cities)
    self._total_distance = 0

    #print(f'genrated stops xy: {self.xy}')
    self.distance_matrix = cdist(self.xy,self.xy,'euclidean').round(0)
    # print(self.distance_matrix.shape)

    self.current_city = np.random.randint(n_cities)
    self.visited_cities = [self.current_city]
    self._array_visited[self.current_city] = 1
    self.remaining_cities = [i for i in range(n_cities)]
    self.cities_list = [i for i in range(n_cities)]
    self.remaining_cities.remove(self.current_city)
    # Define the action space
    self.action_space = gym.spaces.Discrete(n_cities)

    high = np.array(
      [
        np.zeros(4),
        np.zeros(4),
      ],
      dtype=np.int32,
    )

    # Define the observation space
    self.observation_space = gym.spaces.Box(-high, high, dtype=np.int32)
    #self.observation_space = gym.spaces.MultiBinary(n_cities)

    if(self.show_debug_data):
        print(f'Current city in init: {self.current_city}')
        print(f'Remaining city in init: {self.remaining_cities}')
        print(f'Visited city in init: {self.visited_cities}')
        print(f'Stepcounter in init: {self.step_counter}')
        print(f'Observation in init: {self._array_visited}')

  def reset(self):
    self.step_counter = 0
    self._total_distance = 0
    self._array_visited = np.zeros(self.n_cities)
    self.current_city = np.random.randint(self.n_cities)
    self._array_visited[self.current_city] = 1
    self.visited_cities = [self.current_city]
    self.remaining_cities = [i for i in range(self.n_cities)]
    self.remaining_cities.remove(self.current_city)
    
    if(self.show_debug_data):
      print(f'Observation in Reset: {self._array_visited}')
    return self._get_observation()

  def step(self, action):
    done = False
    self.step_counter += 1
    reward = -999

    if(self.show_debug_data):
      print(f'Action in Step(top): {action}')
      print(f'Remaining-City Step(top): {self.remaining_cities}')
      print(f'Observation in step: {self._array_visited}')
    
    if (action in self.remaining_cities):
      if(self.show_debug_data):
        print('Action True')
      
      reward = -self.distance_matrix[self.current_city][action]
      self._total_distance += reward

      self.remaining_cities.remove(action)
      self.visited_cities.append(action)
      self._array_visited[action] = 1
      self.current_city = action

      if (len(self.remaining_cities) == 0):
        startingpoint = self.visited_cities[0]
        self.visited_cities.append(startingpoint)
        reward += -self.distance_matrix[self.current_city][startingpoint]
        done = True
      
      if(self.show_debug_data):
        print(f'Action in step: {action}')
        print(f'Reward in step: {reward}')
        print(f'Current city in step: {self.current_city}')
        print(f'Remaining city in step: {self.remaining_cities}')
        print(f'Visited city in step: {self.visited_cities}')
        print(f'Stepcounter in step: {self.step_counter}')
        print(f'Observation in step: {self._array_visited}')
      
      return self._get_observation(), reward, done, {}
    else:
        return self._get_observation(), reward, False, {}



  def _get_observation(self):
    observationDistance = self.distance_matrix[self.current_city]
    # observation[self.current_city] = 1
    # return observation
    observation = self._array_visited, observationDistance
    
    return np.ravel(observation)

  
  def _test_distance(self,CurrentCity, NextCity):
    return -self.distance_matrix[CurrentCity][NextCity]
    
  def plotCities(self):
    fig, ax = plt.subplots(1, figsize=(7,7))
    fig.suptitle = "Delivery Stops"
    plt.scatter(self.x,self.y)
    xcoord = []
    ycoord = []
    for i in range(0,len(self.visited_cities)):
      xcoord.append(self.x[self.visited_cities[i]])
      ycoord.append(self.y[self.visited_cities[i]])
      if(i == 0):
        ax.annotate("Anfang", xy=(xcoord[i], ycoord[i]), xytext=(xcoord[i]+0.5, ycoord[i]))
      ax.annotate(str(i), xy=(xcoord[i], ycoord[i]), xytext=(xcoord[i]+0.5, ycoord[i]))

    plt.plot(xcoord, ycoord)



In [ ]:
env = TSPEnv(4,False)
samples = []

print('----------')
for i in range(0,env.n_cities):
    samples.append(env.step(i))
print(env.visited_cities)    
env.plotCities()

print(env.step(1))


In [ ]:
#Buffer
import torch
import torch as T
import torch.nn as nn
import torch.optim as optim

class Buffer:
    
    def __init__(self, batch_size, buffer_size, seed):
        
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.rand_generator = np.random.RandomState(seed)
        
        self.buffer = []
        
        
    def append(self, state, action, terminal, reward, next_state):
        """
        Append the next experience.
        
        Args:
            state: the state (torch tensor).
            action: the action (integer).
            terminal: 1 if the next state is the terminal, 0 otherwise.
        
        """
        # delete the first experience if the size is reaching the maximum
        if len(self.buffer) == self.buffer_size:
            del self.buffer[0]
            
        self.buffer.append((torch.tensor(state), torch.tensor(action), torch.tensor(terminal), torch.tensor(reward).float(), torch.tensor(next_state)))
        
    def sample(self):
        """
        Sample from the buffer and return the virtual experience.
        
        Args:
            None
        Returns:
            A list of transition tuples (state, action, terminal, reward, next_state), list length: batch_size
        """
        
        indexs = self.rand_generator.choice(len(self.buffer), size = self.batch_size)
        
        transitions = [self.buffer[idx] for idx in indexs]
        
        return transitions
       
    def get_buffer(self):
        """
        Return the current buffer
        """
        return self.buffer

In [29]:
import gym
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Input, Flatten, Dense

import rl
from rl.memory import SequentialMemory
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

# setup experience replay buffer
memory = SequentialMemory(limit=50000, window_length=1)

# setup the Linear annealed policy with the EpsGreedyQPolicy as the inner policy
policy =  LinearAnnealedPolicy(inner_policy=  EpsGreedyQPolicy(),   # policy used to select actions
                               attr='eps',                          # attribute in the inner policy to vary             
                               value_max=1.0,                       # maximum value of attribute that is varying
                               value_min=0.1,                       # minimum value of attribute that is varying
                               value_test=0.05,                     # test if the value selected is < 0.05
                               nb_steps=10000)                      # the number of steps between value_max and value_min

#Feed-Forward Neural Network Model for Deep Q Learning (DQN)
model = Sequential()
#Input is 1 observation vector, and the number of observations in that vector 
model.add(Input(shape=(1,8)))  
model.add(Flatten())
#Hidden layers with 24 nodes each
model.add(Dense(24, activation='relu'))
model.add(Dense(24, activation='relu'))
#Output is the number of actions in the action space
model.add(Dense(env.action_space.n, activation='linear')) 


#Feed-Forward Neural Network Architecture Summary
print(model.summary())

#Defining DQN Agent for DQN Model
dqn = DQNAgent(model=model,                     # Q-Network model
               nb_actions=env.action_space.n,   # number of actions
               memory=memory,                   # experience replay memory
               nb_steps_warmup=25,              # how many steps are waited before starting experience replay
               target_model_update=1e-2,        # how often the target network is updated
               policy=policy)                   # the action selection policy

# Finally, we configure and compile our agent. 
#We can use built-in tensorflow.keras Adam optimizer and evaluation metrics            
from tensorflow.keras.optimizers import Adam
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae','accuracy'])

#Finally fit and train the agent
history = dqn.fit(env, nb_steps=50000, visualize=False, verbose=2)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 8)                 0         
                                                                 
 dense_27 (Dense)            (None, 24)                216       
                                                                 
 dense_28 (Dense)            (None, 24)                600       
                                                                 
 dense_29 (Dense)            (None, 4)                 100       
                                                                 
Total params: 916
Trainable params: 916
Non-trainable params: 0
_________________________________________________________________
None
Training for 50000 steps ...
     8/50000: episode: 1, duration: 0.088s, episode steps:   8, steps per second:  91, episode reward: -5219.000, mean reward: -652.375 [-999.000, -58.00

c:\Users\fabian.woellenweber\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
c:\Users\fabian.woellenweber\AppData\Local\Programs\Python\Python310\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
c:\Users\fabian.woellenweber\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    27/50000: episode: 4, duration: 0.471s, episode steps:   7, steps per second:  15, episode reward: -4235.000, mean reward: -605.000 [-999.000, -60.000], mean action: 1.429 [0.000, 3.000],  loss: 307982.000000, mae: 166.166534, accuracy: 0.375000, mean_q: 12.423618, mean_eps: 0.997660
    36/50000: episode: 5, duration: 0.041s, episode steps:   9, steps per second: 219, episode reward: -6218.000, mean reward: -690.889 [-999.000, -32.000], mean action: 1.333 [0.000, 3.000],  loss: 280596.680556, mae: 153.572845, accuracy: 0.479167, mean_q: 8.018642, mean_eps: 0.997210
    49/50000: episode: 6, duration: 0.051s, episode steps:  13, steps per second: 255, episode reward: -10214.000, mean reward: -785.692 [-999.000, -32.000], mean action: 1.154 [0.000, 3.000],  loss: 300744.250000, mae: 163.332781, accuracy: 0.288462, mean_q: 1.693867, mean_eps: 0.996220
    57/50000: episode: 7, duration: 0.031s, episode steps:   8, steps per second: 258, episode reward: -5234.000, mean reward: -654.25

c:\Users\fabian.woellenweber\AppData\Local\Programs\Python\Python310\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


    83/50000: episode: 9, duration: 0.073s, episode steps:  23, steps per second: 313, episode reward: -20204.000, mean reward: -878.435 [-999.000, -58.000], mean action: 1.696 [0.000, 3.000],  loss: 300736.954484, mae: 177.083742, accuracy: 0.116848, mean_q: -2.389409, mean_eps: 0.993610
    87/50000: episode: 10, duration: 0.015s, episode steps:   4, steps per second: 265, episode reward: -1238.000, mean reward: -309.500 [-999.000, -45.000], mean action: 1.250 [0.000, 3.000],  loss: 298332.921875, mae: 183.206657, accuracy: 0.031250, mean_q: -7.521979, mean_eps: 0.992395
    93/50000: episode: 11, duration: 0.020s, episode steps:   6, steps per second: 297, episode reward: -3221.000, mean reward: -536.833 [-999.000, -58.000], mean action: 1.833 [0.000, 3.000],  loss: 295482.067708, mae: 187.988480, accuracy: 0.000000, mean_q: -11.663946, mean_eps: 0.991945
    99/50000: episode: 12, duration: 0.022s, episode steps:   6, steps per second: 279, episode reward: -3192.000, mean reward: -

In [ ]:
import numpy as np
import rl
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input
from keras.optimizers import Adam
from collections import deque
import random

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Input(shape=(1,self.state_size)))  
        model.add(Flatten())
        model.add(Dense(24, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        if(len(self.memory) < 100):
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay




In [ ]:
import gym

# create the environment
env = TSPEnv(4)

# create the agent with the number of actions in the environment
state_size = env.observation_space.shape[0]
print(state_size)
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

# training loop
EPISODES = 1000
for episode in range(EPISODES):
    state = env.reset()
    
    state = np.ravel(state)
    print(state)
    for time in range(500):
        # choose an action
        action = agent.act(state)

        # take the action in the environment
        next_state, reward, done, _ = env.step(action)
        
        # remember the experience and update the agent
        agent.remember(state, action, reward, next_state, done)
        agent.replay(32)
        
        # update the current state
        state = next_state
        
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(episode, EPISODES, time, agent.epsilon))
            break


In [ ]:


env = TSPEnv(4,False)


print(env.observation_space.shape[0])
agent = DQNAgent(8, 4)
samples = []
state = env.reset()
agent._build_model()



for i in range(0,env.n_cities):
    action = agent.act(state)
 

# create the agent with 4 actions (corresponding to each city)


# choose an action


# receive a reward and new state
reward = 10
visited_places = [True, True, False, False]
distance_to_next_city = [5, 15, 25, 35]
next_state = np.concatenate((visited_places, distance_to_next_city))
done = False

# remember the experience and update the agent
agent.remember(state, action, reward, next_state, done)
#agent.replay(batch_size=32)




In [ ]:
#Tensorflow DQNAgent
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 32)
        self.fc2 = nn.Linear(32,action_size)
        

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.model = DQN(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.loss_fn = nn.MSELoss()

    def get_action(self, state):
        state = torch.tensor(state, dtype=torch.float32)
        q_values = self.model(state)
        action = q_values.argmax().item()
        return action
    
    def update(self, stateDistance, action, next_state, reward, done):
        stateDistance = torch.tensor(stateDistance, dtype=torch.float32)
        stateVisited = torch.tensor(stateVisited, dtype=torch.int32)
        next_state = torch.tensor(next_state, dtype=torch.float32)
        reward = torch.tensor(reward, dtype=torch.float32)
        done = torch.tensor(done, dtype=torch.float32)
        
        q_values = self.model(state)
        next_q_values = self.model(next_state)
        target = q_values.clone()
        print(f'Target: {target}')
        print(f'Action: {action}')
        target[action] = reward + (1 - done) * next_q_values.max()
        loss = self.loss_fn(q_values, target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


# state[distance / visited] = got from step
# 
# reward = got from step



observation_space = np.array([[1,0,0,0], [0,32,58,60]])
state_size = observation_space.shape[1]
print(state_size)
action_size = 2

agent = DQNAgent(state_size, action_size)


distance = np.array([[0,32,58,60], [1,0,0,0]])
distanceTensor = torch.from_numpy(distance)

print(distanceTensor)
print(distanceTensor[0])
print(distanceTensor[1])


In [ ]:
import gym

num_episodes = 2000

state = env.reset() # get the initial state of the environment
print(f'State in Training: {state}')
# action = agent.get_action(state) # use the agent to select an action
action = 0
print(f'Action in Training: {action}')
next_state, reward, done, info = env.step(action) # take the action in the environment

# state = env.reset()
# # agent.get_action(0)
# # state = env.reset()
# agent.get_action(1)
# state = env.reset()
# agent.get_action(2)


# agent.update(state, action, next_state, reward, done) # update the Q-values
# state = next_state # set the current state to the next state

In [ ]:
import random
import numpy as np
from collections import deque

import torch
import torch as T
import torch.nn as nn
import torch.optim as optim

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the DQN model
class DQN(nn.Module):
  def __init__(self,observation, action, network_arch):
    super().__init__()
    self.num_states = network_arch['num_states']
    self.hidden_units = network_arch['num_hidden_units']
    self.num_actions = network_arch['num_actions']

    self.fc1 = nn.Linear(in_features = self.num_states, out_features = self.hidden_units)
    # self.relu1 = nn.ReLU()
    # self.fc2 = nn.Linear(4, 4)
    # self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(in_features = self.hidden_units, out_features = self.num_actions)
    self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')

  def forward(self, state):
    x = self.fc1(state)
    # x = self.fc2(x)
    action = self.fc3(x)
    return action


# Define the DQNAgent
class DQNAgent:
  def __init__(self, env, epsilon=0.20, epsilon_decay=0.995, epsilon_min=0.01, 
               alpha=1e-3, alpha_decay=0.01, gamma=0.99, memory_size=10000, 
               batch_size=64):

    network_arch = {
    'num_states' : 8,
    'num_hidden_units': 20,
    'num_actions':3
    }

    self.env = env
    self.epsilon = epsilon
    self.epsilon_decay = epsilon_decay
    self.epsilon_min = epsilon_min
    self.alpha = alpha
    self.alpha_decay = alpha_decay
    self.gamma = gamma
    self.memory = deque(maxlen=memory_size)
    self.memory_size = memory_size
    self.batch_size = batch_size
    self.mem_cntr = 0

    # Define the model and the target model
    self.model = DQN(env.observation_space.shape[0], env.action_space.n, network_arch).to(device)
    self.target_model = DQN(env.observation_space.shape[0], env.action_space.n, network_arch).to(device)
    self.target_model.load_state_dict(self.model.state_dict())

    # Define the optimizer
    self.optimizer = optim.Adam(self.model.parameters(), lr=alpha)

    # Define the loss function
    self.loss_fn = nn.MSELoss()


  def remember(self, state, action, reward, next_state, done):
    if(reward >= -3000):
      self.memory.append((state, action, reward, next_state, done))
    else:
      return
  
  def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.append(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

  #Select action
  def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

  def update(self):
    # Don't update if there are not enough samples in the memory
    if len(self.memory) < self.batch_size:
      return

    # Sample a batch from the memory
    samples = random.sample(self.memory, self.batch_size)
  

    # Split the batch into separate variables
    states, actions, rewards, next_states, dones = zip(*samples)

    # Convert variables to tensors and move them to the device
    states = torch.from_numpy(np.vstack(states)).float().to(device)
    print(f'States shape: {states.shape}')
    actions = torch.from_numpy(np.vstack(actions)).long().to(device)
    print(f'Action Shapes: {actions.shape}')
    rewards = torch.from_numpy(np.vstack(rewards)).float().to(device)
    next_states = torch.from_numpy(np.vstack(next_states)).float().to(device)
    dones = torch.from_numpy(np.vstack(dones)).float().to(device)

    # Calculate the Q values for the current states
    q_values = self.model(states)
    # q_values = q_values.gather(1, actions)
  

    # Calculate the Q values for the next states
    # next_q_values = self.target_model(next_states).max(1)[0].unsqueeze(1)
    next_q_values = self.target_model(next_states).max(dim=1)[0].unsqueeze(1)

    # Calculate the target Q values
    target_q_values = rewards + (self.gamma * next_q_values * (1 - dones))

    # Calculate the loss
    loss = self.loss_fn(q_values, target_q_values)

    # Perform backpropagation
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()

    # Update the target model
    for target_param, param in zip(self.target_model.parameters(), self.model.parameters()):
      target_param.data.copy_(param.data * (1 - self.alpha_decay) + target_param.data * self.alpha_decay)

    # Update the epsilon value
    self.epsilon = max(self.epsilon * self.epsilon_decay, self.epsilon_min)
  
  def SaveAgent(self, name):
    torch.save(self.model.state_dict(),'model episode'+name+'pt')
    print('Model saved')

In [ ]:
import gym
import numpy as np


# Instantiate the environment
env = TSPEnv(4)

# Instantiate the agent
agent = DQNAgent(env)

# Set the number of episodes to run
n_episodes = 200

#lists for learning evaluation
scores = []


# Run the episodes
for episode in range(n_episodes):
  # Reset the environment and get the initial state
  state = env.reset()

  # Set the initial reward to 0
  total_reward = 0
  total_steps = 0
  Route = []

  while True:
    # Take an action
    # print(state.shape)
    # print(state)
    action = agent.act(state)

    # Step the environment
    next_state, reward, done, _ = env.step(action)

    # Remember the experience
    agent.remember(state, action, reward, next_state, done)

    # Update the state and the reward
    state = next_state
    total_reward += reward
    total_steps += 1

    # Update the agent
    agent.update()

    # If the episode is done, break the loop
    if done:
      break
  
  scores.append((episode, total_reward, total_steps,env.visited_cities, env._total_distance, env.xy))
  #env.plotCities()
  # Print the total reward for the episode
  print(f"Episode: {episode+1}, Reward: {total_reward}, Steps needed: {total_steps}, Total Distance: {env._total_distance}, Visited Cities: {env.visited_cities}")

  if episode > 1000:
    agent.SaveAgent(str(episode))
  if episode > 5000:
    agent.SaveAgent(str(episode))
  if episode > 10000:
    agent.SaveAgent(str(episode))

  # Open a file handle in write mode
  with open('my_file.txt', 'w') as f:
      # Write the list to the file, one item per line
      f.writelines([f"{item}\n" for item in scores])

